<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/AV_SVM_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
import torch
import re

# Load training data

In [3]:
training_corpus = pd.read_csv("train.csv", encoding='utf-8')

# Pre-process training data

In [4]:
def preprocess(string):
  output = str(string).lower()
  separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

# Prepare data for Distilled Bert
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(training_corpus)

Load BERT Models

In [5]:
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Create BERT Embeddings

In [6]:
SEQ_LENGTH = 128
BATCH_SIZE = 32

def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length, add_special_tokens=True)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)
train_labels = np.array(training_corpus['label'])

# Define Language Model

In [14]:
BERT_ENCODING_DIM = 768

LEARNING_RATE = 5e-5

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768, 1)),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.AveragePooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.AveragePooling1D (pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 766, 128)          512       
                                                                 
 average_pooling1d (Average  (None, 383, 128)          0         
 Pooling1D)                                                      
                                                                 
 conv1d_5 (Conv1D)           (None, 381, 64)           24640     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 190, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_6 (Conv1D)           (None, 188, 32)           6176      
                                                                 
 average_pooling1d_1 (Avera  (None, 94, 32)           

# Train Model

In [15]:
num_samples = train_labels.shape[0]

train_inputs = np.concatenate(bert_embeddings, axis=0).reshape(-1, 768, 1)
print(train_inputs.shape)
model.fit(train_inputs, train_labels, epochs=100, batch_size=128, validation_split=0.1)

(30000, 768, 1)
Epoch 1/100
211/211 [==============================] - 5s 11ms/step - loss: 0.6932 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 2/100
211/211 [==============================] - 1s 6ms/step - loss: 0.6930 - accuracy: 0.5072 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 3/100
211/211 [==============================] - 1s 6ms/step - loss: 0.6928 - accuracy: 0.5132 - val_loss: 0.6929 - val_accuracy: 0.5050
Epoch 4/100
211/211 [==============================] - 1s 7ms/step - loss: 0.6926 - accuracy: 0.5141 - val_loss: 0.6925 - val_accuracy: 0.5243
Epoch 5/100
211/211 [==============================] - 1s 7ms/step - loss: 0.6919 - accuracy: 0.5208 - val_loss: 0.6922 - val_accuracy: 0.5287
Epoch 6/100
211/211 [==============================] - 1s 6ms/step - loss: 0.6915 - accuracy: 0.5268 - val_loss: 0.6919 - val_accuracy: 0.5337
Epoch 7/100
211/211 [==============================] - 1s 6ms/step - loss: 0.6910 - accuracy: 0.5283 - val_loss: 0.6914 - val

# Save Model

In [16]:
model.save("AV_CNN_MODEL")

In [17]:
!zip -r /content/CNN_MODEL.zip /content/AV_CNN_MODEL

  adding: content/AV_SVM_MODEL/ (stored 0%)
  adding: content/AV_SVM_MODEL/fingerprint.pb (stored 0%)
  adding: content/AV_SVM_MODEL/keras_metadata.pb (deflated 91%)
  adding: content/AV_SVM_MODEL/saved_model.pb (deflated 87%)
  adding: content/AV_SVM_MODEL/variables/ (stored 0%)
  adding: content/AV_SVM_MODEL/variables/variables.data-00000-of-00001 (deflated 15%)
  adding: content/AV_SVM_MODEL/variables/variables.index (deflated 63%)
  adding: content/AV_SVM_MODEL/assets/ (stored 0%)
